In [1]:
import mysql.connector as mysql
import os
import pandas as pd

mydb = mysql.connect(host="localhost",
                               user="username",
                               password="password")
mycursor = mydb.cursor()


In [3]:
#inflation data in csv form
df_inf = pd.read_csv('notebook/datasets/DP_LIVE_21112022133845200.csv', index_col=False, delimiter = ',')
df_inf.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,CAN,CPI,TOT,AGRWTH,M,2018-05,2.222222,NaN
1,CAN,CPI,TOT,AGRWTH,M,2018-06,2.453988,NaN
2,CAN,CPI,TOT,AGRWTH,M,2018-07,2.990798,NaN
3,CAN,CPI,TOT,AGRWTH,M,2018-08,2.835249,NaN
4,CAN,CPI,TOT,AGRWTH,M,2018-09,2.217125,NaN


In [10]:
#commodity data in csv form
df_comm = pd.read_csv('notebook/datasets/commodity_prices_august_2021.csv', index_col=False, delimiter = ',')
df_comm.head()

,Month,"Crude oil, average","Crude oil, Brent","Crude oil, Dubai","Crude oil, WTI","Coal, Australian","Coal, South African","Natural gas, US","Natural gas, Europe","Liquefied natural gas, Japan",...,Aluminum,"Iron ore, cfr spot",Copper,Lead,Tin,Nickel,Zinc,Gold,Platinum,Silver
0,1960-01-01,1.63,NaN,1.63,NaN,NaN,NaN,0.14,0.404774,NaN,...,511.471832,11.42,715.40,206.1,2180.4,1631.0,260.8,35.27,83.5,0.9137
1,1960-02-01,1.63,NaN,1.63,NaN,NaN,NaN,0.14,0.404774,NaN,...,511.471832,11.42,728.19,203.7,2180.4,1631.0,244.9,35.27,83.5,0.9137
2,1960-03-01,1.63,NaN,1.63,NaN,NaN,NaN,0.14,0.404774,NaN,...,511.471832,11.42,684.94,210.3,2173.8,1631.0,248.7,35.27,83.5,0.9137
3,1960-04-01,1.63,NaN,1.63,NaN,NaN,NaN,0.14,0.404774,NaN,...,511.471832,11.42,723.11,213.6,2178.2,1631.0,254.6,35.27,83.5,0.9137
4,1960-05-01,1.63,NaN,1.63,NaN,NaN,NaN,0.14,0.404774,NaN,...,511.471832,11.42,684.75,213.4,2162.7,1631.0,253.8,35.27,83.5,0.9137


In [ ]:
#CREATE TABLE FOR INFLATION DATA
from mysql.connector import Error
try:
    conn = mysql.connect(host='localhost', database='mydatabase', user='username', password='password')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS inflation_data;')
        print('Creating table....')
# in the below line please pass the create table statement which you want #to create
        cursor.execute("CREATE TABLE inflation_data(LOCATION varchar(255),INDICATOR varchar(255),SUBJECT varchar(255),MEASURE varchar(255),FREQUENCY varchar(255),TIME varchar(255),Value varchar(255))")
        print("Table is created....")
        #loop through the data frame
        for i, row in df_inf.iterrows():
            #here %S means string values 
            #print(tuple(row)[:-1])
            sql = "INSERT INTO mydatabase.inflation_data VALUES (%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row)[:-1])
            print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
except Error as e:
            print("Error while connecting to MySQL", e)



In [11]:
# Columns to be used for commodity data
# ["Month", 'Crude oil, average', 'Natural gas index', 'Cocoa', 'Orange', 'Gold', 'Sugar, world','Beef', 'Barley']

In [ ]:
#CREATE TABLE FOR COMMODITY DATA
wanted_columns = ["Month", 'Crude oil, average', 'Natural gas index', 'Cocoa', 'Orange', 'Gold', 'Sugar, world','Beef', 'Barley']
try:
    conn = mysql.connect(host='localhost', database='mydatabase', user='username', password='password')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS commodity_data;')
        print('Creating table....')
# in the below line please pass the create table statement which you want #to create
        cursor.execute("CREATE TABLE commodity_data(Month varchar(255),Crude_oil_avg varchar(255),Nat_gas_index varchar(255),\
            Cocoa varchar(255),Orange varchar(255),Gold varchar(255),Sugar_world varchar(255),Beef varchar(255),Barley varchar(255))")
        print("Table is created....")
        #loop through the data frame
        for i, row in df_comm.iterrows():
            #here %S means string values
            
            row_to_insert = [row.get(i) for i in wanted_columns]
            
            sql = "INSERT INTO mydatabase.commodity_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row_to_insert))
            print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
except Error as e:
            print("Error while connecting to MySQL", e)

In [12]:
#UPDATE METHODS

#Insert a new row to preferred table
def put_data_in_table(table_name="commodity_data",\
    row_to_insert=("0","0","0","0","0","0")):

    if table_name == "commodity_data":
        if len(row_to_insert) != 9:
            print("Data to enter is not in the desired length")
            return
    elif table_name == "inflation_data":
        if len(row_to_insert) != 7:
            print("Data to enter is not in the desired length")
            return
    
    conn = mysql.connect(host='localhost', database='mydatabase', user='username', password='password')
    
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        sql = "INSERT INTO mydatabase.{} VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)".format(table_name)
        cursor.execute(sql, row_to_insert)
        print("Record inserted")
        # the connection is not auto committed by default, so we must commit to save our changes
        conn.commit()

    return
    
#Delete rows from table according to the value in the  given column
def remove_record_in_table(table_name="commodity_data", column_of_selection = "Month", value=""):

    conn = mysql.connect(host='localhost', database='mydatabase', user='username', password='password')
    
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        sql = "DELETE FROM mydatabase.{i} WHERE {j} = {k}".format(i=table_name, j=column_of_selection, k=value)
        cursor.execute(sql, row_to_insert)
        print("Record inserted")
        # the connection is not auto committed by default, so we must commit to save our changes
        conn.commit()

    return
